In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler

### Training baseline models ###

In [14]:
# Import data
df = pd.read_csv('../../../../docs/data/cycpeptdb_clean.csv', header=0)
filtered_df = df[df['Permeability'] != -10]

# Select the desired columns
columns = ['TPSA',
           'MolWt',
           'NumHAcceptors', 
           'NumHDonors',
           'NumRotatableBonds',
           'MaxPartialCharge',
           'MinPartialCharge',
           'NHOHCount',
           'NOCount',
           'NumHeteroatoms',
           'NumRotatableBonds',
           'NumSaturatedCarbocycles',
           'NumSaturatedHeterocycles',
           'NumSaturatedRings',
           'RingCount',] + [col for col in filtered_df.columns if col.startswith('fr_')]

# Create the feature matrix and target vector
X = filtered_df[columns].values
X = np.hstack((X, np.ones((X.shape[0], 1))))  # Add a column of ones for the bias term
y = filtered_df['Permeability'].values

# Split the data into train, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

# Define models
models = {
    'Linear Regression': LinearRegression(),
    'Ridge Regression': Ridge(),
    'Lasso Regression': Lasso(),
    'Random Forest Regressor': RandomForestRegressor(random_state=42),
    'Gradient Boosting Regressor': GradientBoostingRegressor(random_state=42)
}

# Initialize a dictionary to store results
results = {}

# Train and evaluate each model
for name, model in models.items():
    # Train the model
    model.fit(X_train, y_train)
    
    # Predict on validation set
    y_val_pred = model.predict(X_val)
    
    # Calculate metrics
    mse = mean_squared_error(y_val, y_val_pred)
    mae = mean_absolute_error(y_val, y_val_pred)
    r2 = r2_score(y_val, y_val_pred)
    
    # Store the results
    results[name] = {"MSE": mse, "MAE": mae, "R2": r2}

# Print the results
for name, metrics in results.items():
    print(f"{name}:")
    print(f"  MSE: {metrics['MSE']}")
    print(f"  MAE: {metrics['MAE']}")
    print(f"  R2: {metrics['R2']}")
    print()

# Evaluate the best model on the test set
# Here, we assume Random Forest Regressor is the best model based on validation performance
best_model = models["Random Forest Regressor"]
y_test_pred = best_model.predict(X_test)
test_mse = mean_squared_error(y_test, y_test_pred)
test_mae = mean_absolute_error(y_test, y_test_pred)
test_r2 = r2_score(y_test, y_test_pred)

print("Test set evaluation (Random Forest Regressor):")
print(f"  MSE: {test_mse}")
print(f"  MAE: {test_mae}")
print(f"  R2: {test_r2}")


Linear Regression:
  MSE: 0.4334098226801136
  MAE: 0.4967782339497994
  R2: 0.24701883226116284

Ridge Regression:
  MSE: 0.4337200576027151
  MAE: 0.49671018986704335
  R2: 0.24647984804329415

Lasso Regression:
  MSE: 0.5759167290588226
  MAE: 0.5806486519634247
  R2: -0.0005644276482186239

Random Forest Regressor:
  MSE: 0.22025048068648717
  MAE: 0.34223361492609583
  R2: 0.6173495489400664

Gradient Boosting Regressor:
  MSE: 0.2835014709955166
  MAE: 0.39822356584884855
  R2: 0.5074609353202437

Test set evaluation (Random Forest Regressor):
  MSE: 0.2138638895297967
  MAE: 0.3316172483329435
  R2: 0.6350826022761533


### Printing weights ###

In [15]:
# Initialize a dictionary to store feature weights
feature_weights = {}

# Feature names (excluding the bias term)
feature_names = columns + ['Bias Term']

# Extract feature weights or importances for each model
for name, model in models.items():
    if hasattr(model, 'coef_'):
        # For linear models
        weights = model.coef_
    elif hasattr(model, 'feature_importances_'):
        # For tree-based models
        weights = model.feature_importances_
    else:
        # Skip models that do not have interpretable feature importances
        continue
    
    # Create a dictionary of feature names and their corresponding weights
    feature_weights[name] = {feature: weight for feature, weight in zip(feature_names, weights)}
    
    # Sort the dictionary by absolute weight values in descending order
    feature_weights[name] = dict(sorted(feature_weights[name].items(), key=lambda item: abs(item[1]), reverse=True))

# Print the feature weights for each model
for name, weights in feature_weights.items():
    print(f"Feature weights for {name}:")
    for feature, weight in weights.items():
        print(f"  {feature}: {weight}")
    print()

Feature weights for Linear Regression:
  fr_Ar_NH: -3712503578022.287
  fr_Nhpyrrole: 3699392287875.923
  NumSaturatedRings: 2432919416067.335
  NumSaturatedHeterocycles: -2382732709422.2095
  NumRotatableBonds: 1678152541899.4414
  NumHDonors: 1329751714701.392
  fr_ketone_Topliss: -1233777624381.3367
  fr_ketone: 1233777624381.3286
  fr_Al_COO: -721061181993.5986
  NHOHCount: -720610901282.822
  NumSaturatedCarbocycles: -707616097538.7589
  NumHeteroatoms: 625648314895.0669
  fr_NH1: -537389407652.845
  NumHAcceptors: -487401552884.7
  fr_amide: -392261055543.9801
  fr_COO2: 371849068041.24524
  fr_COO: 330135644990.4816
  NOCount: 252934560052.52744
  fr_C_O: 166884465015.36768
  fr_C_O_noCOO: -166848781256.6166
  fr_Al_OH: -150909285117.4111
  fr_alkyl_carbamate: -140965518621.24072
  fr_C_S: 91462939035.6796
  fr_Ar_COO: 72949083090.74513
  fr_amidine: -57271429758.91031
  fr_N_O: -51269196951.00114
  fr_halogen: -46704981268.55118
  fr_azo: 46320593474.45642
  fr_phenol: -4405730